In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# based on http://www.rows.ws/jpihack/ and https://github.com/wannamak/edmtools 


from datetime import datetime 
from datetime import timedelta
import struct
import sys
import os

import config

In [9]:
with open('U231218.JPI', "rb") as f: 
    
    engData = f.read()

    for i in engData.split(b'\r\n'):
        print(i)
'''
    while engData:
        print(engData);
        engData = f.readline()
'''

'''
$A = configured limits:
VoltsHi*10,VoltsLo*10,DIF,CHT,CLD,TIT,OilHi,OilLo
"$A,305,230,500,415,60,1650,230,90*7F"
'''

'''
$F = Fuel flow config and limits
empty,full,warning,kfactor,kfactor
**NEW** 'warning' field seems to be AUX tank capacity instead
"$F,0,999, 0,2950,2950*53"
'''

'''
$T = timestamp of download, fielded (Times are UTC)
MM,DD,YY,hh,mm,?? maybe some kind of seq num but not strictly sequential?
"$T, 5,13, 5,23, 2, 2222*65"
'''

'''
$C = config info (only partially known)
model#,feature flags lo, feature flags hi, unknown flags,firmware version
"$C, 700,63741, 6193, 1552, 292*58"

The feature flags is a 32 bit set of flags as follows:

// -m-d fpai r2to eeee eeee eccc cccc cc-b
//
// e = egt (up to 9 cyls)
// c = cht (up to 9 cyls)
// d = probably cld
// b = bat
// o = oil
// t = tit1
// 2 = tit2
// a = OAT
// f = fuel flow
// r = CDT (also CARB - not distinguished in the CSV output)
// i = IAT
// m = MAP
// p = RPM
// *** e and c may be swapped (but always exist in tandem)
// *** d and b may be swapped (but seem to exist in tandem)
// *** m, p and i may be swapped among themselves, haven't seen
//     enough independent examples to know for sure.

**NEW** I think d (bit 29) is ENGINE TEMPS UNIT (1=F) as I compared two saved configurations with this change and it was the only difference


'''

'''
**NEW**
$P = CARB  (wild guess)
1 – lowest Fuel Flow filter
2 – medium Fuel Flow filter
3 – highest Fuel Flow filter        
'''

'''
$D = flight info
flight#, length of flight's data in 16 bit words
"$D,  227, 3979*57"

there's one line per flight
'''

'''
unknown meaning
$H = 0 
$L = last header record
"$L, 49*4D"
'''


'''
The flight header follows immediately after the $L record, and is as follows:

struct flightheader {
ushort flightnumber;  // matches what's in the $D record
ulong flags;          // matches the "feature flags" in the $C record
ushort unknown;       // may contain flags about what units (e.g. F vs. C)
ushort unknown2[7];   // **NEW** I hope there is some sort of a header version somewhere!
ushort interval_secs; // record interval, in seconds (usually 6)
datebits dt;          // date as fielded bits, see struct below
timebits tm;          // time as fielded bits, see struct below
};

The datebits/timebits are bit fields as follows:

// pack a date into 16 bits
struct datebits {
unsigned day:5;
unsigned mon:4;
unsinged year:7;
};

struct timebits {
unsigned secs:5;      // #secs / 2 is stored
unsigned mins:6;
unsigned hrs:5;
};
        '''

'''
        **NEW**
        The overall record structure quite differs from the original, simpler than expected actually.

        struct record:
        byte somevalue
        short decodeflags[2]
        // decodeflags[0] == decodeflags[1]
        byte repeatcount
        // If repeat count is non-zero, output the current data set again (incrementing timestamp) and then continue processing.

        // For each bit set in decodeflags: fieldflags[x], and signflags[x] will exist
        byte fieldflags[16];   
        byte signflags[16];

        // then there is a value for each bit set in fieldflags
        // each value is stored as a difference with the previous value, the initial value being 0xF0 or 0 depending on the field
        // some fields are stored as two bytes (EGT, RPM, etc)
'''



b'$U,N8916M_*23'
b'$A,160,120,500,400, 60,1650,230, 90*78'
b'$F,0, 50,  0,   0,   0*4F'
b'$T,12,19,23, 3,17, 1171*6D'
b'$C, 700,63741, 4113,  144, 291*43'
b'$D,  915, 2522*5E'
b'$D,  916, 2186*57'
b'$D,  917, 2168*56'
b'$D,  918,  410*41'
b'$D,  919,  117*42'
b'$D,  920, 7707*58'
b'$D,  921,  435*4C'
b'$D,  922,   24*5B'
b'$D,  923, 8057*56'
b'$D,  924, 1947*50'
b'$D,  925, 1854*52'
b'$D,  926,   80*51'
b'$D,  927, 1488*5D'
b'$D,  928,  421601*4C'
b'$D, 1010, 1271*41'
b'$D, 1011, 6040*47'
b'$D, 1012, 1326*40'
b'$D, 1013, 4156*41'
b'$D, 1014,   45*41'
b'$D, 1015,   24*47'
b'$D, 1016, 1691*4D'
b'$D, 1017, 1235*46'
b'$D, 1018,  135*5B'
b'$D, 1019, 7887*4D'
b'$D, 1020,  199*56'
b'$D, 1021,   47*45'
b'$D, 1022,   24*43'
b'$D, 1023, 7648*49'
b'$D, 1024, 1426*42'
b'$D, 1025, 7516*47'
b'$D, 1026,   82*4B'
b'$D, 1027,   45*41'
b'$D, 1028,   24*49'
b'$D, 1029, 5011*4B'
b'$L,142*57'
b'\x01\x80\x80\x03\x02\x02\x02\x01\x01\x00\x02\x02\x02\x02\x01\x80\x00\x02\x83\x01\x01\x00  \x05\x05\x02\x02\x00\x8

'\n        **NEW**\n        The overall record structure quite differs from the original, simpler than expected actually.\n\n        struct record:\n        byte somevalue\n        short decodeflags[2]\n        // decodeflags[0] == decodeflags[1]\n        byte repeatcount\n        // If repeat count is non-zero, output the current data set again (incrementing timestamp) and then continue processing.\n\n        // For each bit set in decodeflags: fieldflags[x], and signflags[x] will exist\n        byte fieldflags[16];   \n        byte signflags[16];\n\n        // then there is a value for each bit set in fieldflags\n        // each value is stored as a difference with the previous value, the initial value being 0xF0 or 0 depending on the field\n        // some fields are stored as two bytes (EGT, RPM, etc)\n'